In [45]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from statsmodels.api import OLS, add_constant
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from statsmodels.api import add_constant

In [46]:
!pip install autograd
from autograd import grad
from autograd import numpy as anp

In [47]:
# Parameters
num_individuals = 1000
num_snps = 5000
heritability = 0.8  # Proportion of variance explained by genetics

# Set random seed for reproducibility
np.random.seed(42)

# Generate Synthetic Genotype Data
maf = np.random.uniform(0.05, 0.5, num_snps)
genotypes = np.random.binomial(2, maf, (num_individuals, num_snps))

# Simulate Phenotype Data
num_causal_snps = 200
causal_indices = np.random.choice(num_snps, num_causal_snps, replace=False)
raw_effect_sizes = np.random.normal(0, 0.5, num_causal_snps)  # Initial random effect sizes

# Calculate initial genetic contribution
initial_genetic_contribution = genotypes[:, causal_indices].dot(raw_effect_sizes)

# Scale effect sizes to ensure exact heritability
genetic_variance_required = heritability * initial_genetic_contribution.var()
scaling_factor = np.sqrt(genetic_variance_required / initial_genetic_contribution.var())
scaled_effect_sizes = raw_effect_sizes * scaling_factor
genetic_contribution = genotypes[:, causal_indices].dot(scaled_effect_sizes)

# Calculate environmental contribution
environmental_variance_required = (1 - heritability) * genetic_contribution.var() / heritability
environmental_contribution = np.random.normal(0, np.sqrt(environmental_variance_required), num_individuals)

# Combine genetic and environmental contributions to form phenotypes
phenotypes = genetic_contribution + environmental_contribution

# Output for verification
print("Calculated Heritability:", genetic_contribution.var() / phenotypes.var())

Calculated Heritability: 0.7937975933712973


In [48]:
# Splitting data into training and validation sets
indices = np.arange(num_individuals)
np.random.shuffle(indices)
split_point = num_individuals // 2

training_indices = indices[:split_point]
validation_indices = indices[split_point:]

training_genotypes = genotypes[training_indices]
training_phenotypes = phenotypes[training_indices]

validation_genotypes = genotypes[validation_indices]
validation_phenotypes = phenotypes[validation_indices]

# Use separate scalers for genotype data to ensure no leakage between training and validation sets
scaler_genotypes_train = StandardScaler()
scaler_genotypes_valid = StandardScaler()

training_genotypes_scaled = scaler_genotypes_train.fit_transform(training_genotypes)
validation_genotypes_scaled = scaler_genotypes_valid.fit_transform(validation_genotypes)

# Use separate scalers for phenotype data as well
scaler_phenotypes_train = StandardScaler()
scaler_phenotypes_valid = StandardScaler()

training_phenotypes_scaled = scaler_phenotypes_train.fit_transform(training_phenotypes.reshape(-1, 1)).flatten()
validation_phenotypes_scaled = scaler_phenotypes_valid.fit_transform(validation_phenotypes.reshape(-1, 1)).flatten()

In [49]:
import numpy as np
from autograd import grad
import autograd.numpy as anp

def perform_ols(X_scaled, y_scaled, num_iterations=10000, learning_rate=0.0001):
    """
    Perform OLS regression using gradient descent.
    
    Args:
    - X_scaled: Scaled genotype data.
    - y_scaled: Scaled phenotype data.
    - num_iterations: Number of iterations for gradient descent.
    - learning_rate: Learning rate for gradient descent.
    
    Returns:
    - beta_coefficients: Estimated regression coefficients.
    """
    num_individuals, num_snps = X_scaled.shape
    X_with_bias = anp.hstack([anp.ones((num_individuals, 1)), X_scaled])
    
    # Initialize coefficients
    beta_coefficients = np.zeros(num_snps + 1)
    
    # Define the objective function for OLS
    def objective(beta):
        predictions = anp.dot(X_with_bias, beta)
        return anp.mean((y_scaled - predictions) ** 2)
    
    # Compute the gradient using autograd
    gradient = grad(objective)
    
    # Perform gradient descent
    for _ in range(num_iterations):
        grads = gradient(beta_coefficients)
        beta_coefficients -= learning_rate * grads
    
    return beta_coefficients

def perform_lasso(X_scaled, y_scaled, lambda_, num_iterations=10000, learning_rate=0.0001):
    """
    Perform Lasso regression using gradient descent with subgradient for L1 term.
    
    Args:
    - X_scaled: Scaled genotype data.
    - y_scaled: Scaled phenotype data.
    - lambda_: Regularization strength for Lasso.
    - num_iterations: Number of iterations for gradient descent.
    - learning_rate: Learning rate for gradient descent.
    
    Returns:
    - beta_coefficients: Estimated regression coefficients.
    """
    num_individuals, num_snps = X_scaled.shape
    X_with_bias = anp.hstack([anp.ones((num_individuals, 1)), X_scaled])
    
    # Initialize coefficients
    beta_coefficients = np.zeros(num_snps + 1)
    
    # Define the objective function for Lasso
    def objective(beta):
        predictions = anp.dot(X_with_bias, beta)
        return anp.mean((y_scaled - predictions) ** 2) + lambda_ * anp.sum(anp.abs(beta))
    
    # Compute the gradient using autograd
    gradient = grad(objective)
    
    # Perform gradient descent
    for _ in range(num_iterations):
        grads = gradient(beta_coefficients)
        beta_coefficients -= learning_rate * grads
    
    return beta_coefficients

In [50]:
def call_train(method, X, y, lambda_strength=0.1):
    """
    Wrapper function to perform either OLS or Lasso regression based on the method specified.
    
    Args:
    - method: str, 'OLS' or 'Lasso' indicating the type of regression.
    - X: ndarray, the scaled genotype data matrix.
    - y: ndarray, the scaled phenotype data vector.
    - lambda_strength: float, the regularization strength for Lasso. Default is 0.1.

    Returns:
    - beta: ndarray, the estimated regression coefficients.
    """
    print(f"Fitting {method}...")
    if method == 'OLS':
        beta = perform_ols(X, y)
    elif method == 'Lasso':
        # Validate that lambda_strength is provided for Lasso
        if lambda_strength is None:
            raise ValueError("Lambda strength must be provided for Lasso regression.")
        beta = perform_lasso(X, y, lambda_strength)
    else:
        raise ValueError("Unsupported regression method specified.")

    # Identify and report the top 10 coefficients
    top_indices = np.argsort(-np.abs(beta))[:10]
    print(f"Top 10 largest beta coefficients from {method}:")
    for index in top_indices:
        print(f"Beta[{index}] = {beta[index]}")

    return beta

# Example usage within the script
print("Training:")

# Regularization strength for Lasso (if needed)
lambda_strength = 0.01

# Perform OLS and Lasso regression, and print the top 10 coefficients
beta_ols = call_train('OLS', training_genotypes_scaled, training_phenotypes_scaled)
beta_lasso = call_train('Lasso', training_genotypes_scaled, training_phenotypes_scaled, lambda_strength)

Training:
Fitting OLS...
Top 10 largest beta coefficients from OLS:
Beta[3320] = 0.022778990436936847
Beta[1801] = 0.021355685942634338
Beta[716] = 0.02110980602387615
Beta[1728] = 0.0175112512287216
Beta[3783] = 0.016786667362839344
Beta[3771] = -0.016570941709073056
Beta[3295] = 0.016178125260421086
Beta[4692] = -0.015186389538063816
Beta[3388] = 0.015079971423144852
Beta[3318] = 0.014464007084116506
Fitting Lasso...
Top 10 largest beta coefficients from Lasso:
Beta[3320] = 0.04430302078813965
Beta[1801] = 0.043988053644574886
Beta[716] = 0.04081591890928134
Beta[3771] = -0.030009884572536075
Beta[4692] = -0.029046928762201593
Beta[1728] = 0.02897453761318064
Beta[3783] = 0.027692136619178204
Beta[3295] = 0.02720606068571836
Beta[1936] = 0.025621110603992178
Beta[1464] = -0.024878752997785747


In [51]:
def calculate_and_normalize_prs(genotypes_scaled, beta):
    """Calculate and robustly normalize the Polygenic Risk Score."""
    # Calculate PRS
    prs = genotypes_scaled.dot(beta[1:]) + beta[0]
    
    # Robust normalization using median and IQR
    median = np.median(prs)
    iqr = np.percentile(prs, 75) - np.percentile(prs, 25)
    normalized_prs = (prs - median) / iqr
    
    return normalized_prs

def print_detailed_statistics(scores, title):
    """Print detailed statistics for PRS scores."""
    sorted_scores = sorted(scores)
    print(f"{title}:")
    print("Top 3 values:", sorted_scores[-3:])
    print("Bottom 3 values:", sorted_scores[:3])
    print(f"Mean: {np.mean(scores):.4f}, Median: {np.median(scores):.4f}, Std Dev: {np.std(scores):.4f}")
    print(f"25th percentile: {np.percentile(scores, 25):.4f}, 75th percentile: {np.percentile(scores, 75):.4f}\n")

# Processing training data
normalized_training_prs_ols = calculate_and_normalize_prs(training_genotypes_scaled, beta_ols)
normalized_training_prs_lasso = calculate_and_normalize_prs(training_genotypes_scaled, beta_lasso)

print("Training:")
print_detailed_statistics(normalized_training_prs_ols, "Normalized PRS (OLS)")
print_detailed_statistics(normalized_training_prs_lasso, "Normalized PRS (Lasso)")

# Processing validation data
normalized_validation_prs_ols = calculate_and_normalize_prs(validation_genotypes_scaled, beta_ols)
normalized_validation_prs_lasso = calculate_and_normalize_prs(validation_genotypes_scaled, beta_lasso)

print("\nValidation:")
print_detailed_statistics(normalized_validation_prs_ols, "Normalized PRS (OLS)")
print_detailed_statistics(normalized_validation_prs_lasso, "Normalized PRS (Lasso)")

Training:
Normalized PRS (OLS):
Top 3 values: [2.0698673198016775, 2.0831070534623755, 2.7329564103079345]
Bottom 3 values: [-2.1239144462093122, -2.062754947821718, -1.9450641451518034]
Mean: -0.0223, Median: 0.0000, Std Dev: 0.7776
25th percentile: -0.5424, 75th percentile: 0.4576

Normalized PRS (Lasso):
Top 3 values: [2.1191609586109417, 2.150526853942557, 2.8062773935961847]
Bottom 3 values: [-2.194768608879356, -2.09151866487082, -1.9613706985541761]
Mean: -0.0002, Median: 0.0000, Std Dev: 0.7904
25th percentile: -0.5030, 75th percentile: 0.4970


Validation:
Normalized PRS (OLS):
Top 3 values: [1.8729152751017661, 2.0710390292404166, 2.5236200391524237]
Bottom 3 values: [-2.502635026104092, -2.314353100652104, -2.144212357248877]
Mean: -0.0258, Median: 0.0000, Std Dev: 0.7338
25th percentile: -0.5142, 75th percentile: 0.4858

Normalized PRS (Lasso):
Top 3 values: [1.9471982091208453, 2.063203902734755, 2.2531861128469415]
Bottom 3 values: [-2.3246137497238197, -2.277826919999820

In [52]:
def calculate_stats(prs_scaled, phenotypes_scaled):
    """Calculate and print the correlation and variance explained for a given set of PRS and phenotypes."""
    correlation, _ = pearsonr(prs_scaled, phenotypes_scaled)
    variance_explained = correlation ** 2
    return correlation, variance_explained

# Calculate statistics for the training data
training_correlation_ols, training_variance_explained_ols = calculate_stats(normalized_training_prs_ols, training_phenotypes_scaled)
training_correlation_lasso, training_variance_explained_lasso = calculate_stats(normalized_training_prs_lasso, training_phenotypes_scaled)

print("Training Statistics:")
print(f"OLS Correlation: {training_correlation_ols:.6f}, Variance Explained: {training_variance_explained_ols:.6f}")
print(f"Lasso Correlation: {training_correlation_lasso:.6f}, Variance Explained: {training_variance_explained_lasso:.6f}")

# Calculate statistics for the validation data
validation_correlation_ols, validation_variance_explained_ols = calculate_stats(normalized_validation_prs_ols, validation_phenotypes_scaled)
validation_correlation_lasso, validation_variance_explained_lasso = calculate_stats(normalized_validation_prs_lasso, validation_phenotypes_scaled)

print("\nValidation Statistics:")
print(f"OLS Correlation: {validation_correlation_ols:.6f}, Variance Explained: {validation_variance_explained_ols:.6f}")
print(f"Lasso Correlation: {validation_correlation_lasso:.6f}, Variance Explained: {validation_variance_explained_lasso:.6f}")

Training Statistics:
OLS Correlation: 1.000000, Variance Explained: 1.000000
Lasso Correlation: 0.999254, Variance Explained: 0.998509

Validation Statistics:
OLS Correlation: 0.184523, Variance Explained: 0.034049
Lasso Correlation: 0.240438, Variance Explained: 0.057810
